In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pyarrow
from datetime import datetime
from meteostat import Point, Daily, Stations, Monthly

#https://meteostat.net/en/blog/obtain-weather-data-any-location-python

In [2]:
!pip install meteostat


In [2]:
cwd = os.getcwd()
cwd
pd.set_option('display.max_columns', None)

In [3]:
point_elevations = pd.read_csv('pollution_point_source_elevations.csv', low_memory=False)
school_elevations = pd.read_parquet('school_elevations.parquet', engine='pyarrow')

In [3]:
joined = pd.read_parquet('joined_data_fixed_counties_10-6-22.parquet', engine='pyarrow')

In [159]:
joined.head()

,year,cdscode,school_county,school_zip,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,open_close_dates_valid,Taxable Year,Zip Code,CA_AGI_per_returns,Total Tax Liability,school_county_v2,school_region_name,nearby_point_source_count
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2010.0,20102070109728,Madera,93610,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0
1,2010.0,20102072030187,Madera,93610,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0
2,2010.0,20651930000000,Madera,93610,NaT,NaT,37.122449,-120.264870,1.531352,72.77,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2358.38241919395,-37.54028477906731,NaN,NaN,1.531352,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,0,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,0.0
3,2010.0,20651930113076,Madera,93610,2007-08-21,NaT,37.131113,-120.242025,1.531352,77.12,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2892.9075458251764,11.831222108044384,NaN,NaN,1.531352,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,0.0
4,2010.0,20651936023907,Madera,93610,1980-07-01,NaT,37.115243,-120.267770,1.531352,70.66,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2004.5665433091058,-57.72695665034661,NaN,NaN,1.531352,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0


In [165]:
joined.shape[0]

2983111

In [166]:
type(joined['school_open_date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [162]:
type(joined['school_lon'][0])

numpy.float64

In [27]:
type(joined['school_lat'][0])

numpy.float64

In [58]:
type(joined['year'][0])

numpy.float64

In [59]:
joined['year']

0          2010.0
1          2010.0
2          2010.0
3          2010.0
4          2010.0
            ...  
2983106    2016.0
2983107    2016.0
2983108    2016.0
2983109    2016.0
2983110    2016.0
Name: year, Length: 2983111, dtype: float64

In [7]:
joined.dtypes

year                         float64
cdscode                       object
school_county                 object
school_zip                    object
school_region_name            object
school_open_date              object
school_closed_date            object
school_lat                    object
school_lon                    object
school_wspd                  float64
school_elevation_m           float64
year_month                    object
total_population              object
total_population_male         object
total_population_female       object
population_0_4                object
population_0_4_male           object
population_0_4_female         object
population_5_9                object
population_5_9_male           object
population_5_9_female         object
population_10_14              object
population_10_14_male         object
population_10_14_female       object
population_15_19              object
population_15_19_male         object
population_15_19_female       object
p

In [18]:
type(elevations['point_source_lat'][0])

numpy.float64

In [19]:
type(elevations['point_source_lon'][0])

numpy.float64

In [20]:
type(elevations['elevation_meters'][0])

numpy.float64

# Convert joined dataset lat/lons to floats
Don't need to run anymore

In [8]:
def str_to_float(string):
    string = string
    try:
        string = float(string)
    except:
        print(string)
    return string

In [9]:
# converting lat column to floats
joined['school_lat'] = joined['school_lat'].apply(str_to_float)

In [10]:
# converting lon column to floats
joined['school_lon'] = joined['school_lon'].apply(str_to_float)

# Data Cleaning!
* post- 'joined_data_fixed_counties_10-6-22.parquet' which contains fixed counties and income

In [167]:
# Replace Nat Type values with Nones
joined.school_open_date.replace({np.nan: None}, inplace = True)
joined.school_closed_date.replace({np.nan: None}, inplace = True)

In [182]:
joined.head()

,year,cdscode,school_county,school_zip,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,open_close_dates_valid,Taxable Year,Zip Code,CA_AGI_per_returns,Total Tax Liability,school_county_v2,school_region_name,nearby_point_source_count
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2010.0,20102070109728,Madera,93610,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0
1,2010.0,20102072030187,Madera,93610,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0
2,2010.0,20651930000000,Madera,93610,1990-01-01,2025-01-01,37.122449,-120.264870,1.531352,72.77,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2358.38241919395,-37.54028477906731,NaN,NaN,1.531352,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,0,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,0.0
3,2010.0,20651930113076,Madera,93610,2007-08-21,2025-01-01,37.131113,-120.242025,1.531352,77.12,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2892.9075458251764,11.831222108044384,NaN,NaN,1.531352,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,0.0
4,2010.0,20651936023907,Madera,93610,1980-07-01,2025-01-01,37.115243,-120.267770,1.531352,70.66,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2004.5665433091058,-57.72695665034661,NaN,NaN,1.531352,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0


In [164]:
joined.dtypes

year                                float64
cdscode                               int64
school_county                        object
school_zip                            int32
school_open_date             datetime64[ns]
school_closed_date           datetime64[ns]
school_lat                          float64
school_lon                          float64
school_wspd                         float64
school_elevation_m                  float64
year_month                           object
total_population                     object
total_population_male                object
total_population_female              object
population_0_4                       object
population_0_4_male                  object
population_0_4_female                object
population_5_9                       object
population_5_9_male                  object
population_5_9_female                object
population_10_14                     object
population_10_14_male                object
population_10_14_female         

In [183]:
# testing
#joined['school_closed_date'][2] is None

In [7]:
def change_dt_format(date_string, format="%m/%d/%Y"):
    formatted_date = datetime.strptime(date_string, format)
    return formatted_date

In [180]:
# replace "No Data" with dates for start and end
def replace_no_data(date_string, start_end):
    if date_string is None:
        if start_end == 'start':
            date = change_dt_format('01/01/1990')
        else:
            date = change_dt_format('01/01/2025')
    else:
        date = date_string
        
    return date


In [181]:
joined['school_open_date'] = joined['school_open_date'].apply(lambda x: replace_no_data(x, 'start'))
joined['school_closed_date'] = joined['school_closed_date'].apply(lambda x: replace_no_data(x, 'end'))

In [191]:
#joined['year_month'].value_counts()

In [186]:
# 871 nulls in year_month col; dropping
# year_month comes from the wind data. When we joined that to the schools, 
# those were the school zips that weren't found in the wind data

#joined[joined['year_month'].isnull()]

In [187]:
joined = joined[joined['year_month'].notna()]

In [140]:
joined['school_open_date'].isnull().sum()

0

In [141]:
joined['school_closed_date'].isnull().sum()

0

In [188]:
joined['year_month'].isnull().sum()

0

In [189]:
# checks out after dropped
joined.shape[0]

2982240

In [ ]:
joined['year_month'] = joined['year_month'] 

In [192]:
joined.to_parquet("joined_intermediate_10-10-22.parquet")

In [15]:
joined = pd.read_parquet("joined_intermediate_10-10-22.parquet", engine='pyarrow')

In [13]:
joined.head()

,year,cdscode,school_county,school_zip,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,open_close_dates_valid,Taxable Year,Zip Code,CA_AGI_per_returns,Total Tax Liability,school_county_v2,school_region_name,nearby_point_source_count,year_month_date
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2010.0,20102070109728,Madera,93610,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0,2010-04-01
1,2010.0,20102072030187,Madera,93610,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0,2010-04-01
2,2010.0,20651930000000,Madera,93610,1990-01-01,2025-01-01,37.122449,-120.264870,1.531352,72.77,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2358.38241919395,-37.54028477906731,NaN,NaN,1.531352,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,0,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,0.0,2010-04-01
3,2010.0,20651930113076,Madera,93610,2007-08-21,2025-01-01,37.131113,-120.242025,1.531352,77.12,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2892.9075458251764,11.831222108044384,NaN,NaN,1.531352,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,0.0,2010-04-01
4,2010.0,20651936023907,Madera,93610,1980-07-01,2025-01-01,37.115243,-120.267770,1.531352,70.66,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2004.5665433091058,-57.72695665034661,NaN,NaN,1.531352,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0,2010-04-01


In [16]:

# joined2['school_open_date'] = joined2['school_open_date'].apply(lambda x: change_dt_format(x))
# joined2['school_closed_date'] = joined2['school_closed_date'].apply(lambda x: change_dt_format(x))
joined['year_month'] = joined['year_month'].apply(lambda x: change_dt_format(x, format="%Y-%m"))

In [17]:
#Change datetime to just month
#joined['year_month'] = pd.to_datetime(joined['year_month']).dt.to_period('M')

In [18]:
joined.head()

,year,cdscode,school_county,school_zip,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,open_close_dates_valid,Taxable Year,Zip Code,CA_AGI_per_returns,Total Tax Liability,school_county_v2,school_region_name,nearby_point_source_count
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2010.0,20102070109728,Madera,93610,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04-01,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0
1,2010.0,20102072030187,Madera,93610,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04-01,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0
2,2010.0,20651930000000,Madera,93610,1990-01-01,2025-01-01,37.122449,-120.264870,1.531352,72.77,2010-04-01,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2358.38241919395,-37.54028477906731,NaN,NaN,1.531352,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,0,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,0.0
3,2010.0,20651930113076,Madera,93610,2007-08-21,2025-01-01,37.131113,-120.242025,1.531352,77.12,2010-04-01,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2892.9075458251764,11.831222108044384,NaN,NaN,1.531352,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,0.0
4,2010.0,20651936023907,Madera,93610,1980-07-01,2025-01-01,37.115243,-120.267770,1.531352,70.66,2010-04-01,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2004.5665433091058,-57.72695665034661,NaN,NaN,1.531352,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0


In [108]:
# joined2['school_open_date'] = pd.to_datetime(joined2['school_open_date'], format='%m/%d/%Y').dt.strftime('%m/%d/%Y')
# joined2['school_closed_date'] = pd.to_datetime(joined2['school_closed_date'], format='%m/%d/%Y').dt.strftime('%m/%d/%Y')
# joined2['year_month'] = pd.to_datetime(joined2['year_month'], format='%Y-%m').dt.strptime('%m/%Y')


In [109]:
# joined2['school_open_date'] = pd.to_datetime(joined2['school_open_date'], format='%m/%d/%Y')
# joined2['school_closed_date'] = pd.to_datetime(joined2['school_closed_date'], format='%m/%d/%Y')
# joined2['year_month'] = pd.to_datetime(joined2['year_month'], format='%m/%Y')

In [218]:
#joined2.dtypes

In [106]:
#joined2.to_parquet("joined_datetime_replaced_strings_10-10-22.parquet")

In [23]:
joined.head()

,year,cdscode,school_county,school_zip,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,open_close_dates_valid,Taxable Year,Zip Code,CA_AGI_per_returns,Total Tax Liability,school_county_v2,school_region_name,nearby_point_source_count,open_dates_valid
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2010.0,20102070109728,Madera,93610,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04-01,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0,0
1,2010.0,20102072030187,Madera,93610,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04-01,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0,0
2,2010.0,20651930000000,Madera,93610,1990-01-01,2025-01-01,37.122449,-120.264870,1.531352,72.77,2010-04-01,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2358.38241919395,-37.54028477906731,NaN,NaN,1.531352,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,0,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,0.0,0
3,2010.0,20651930113076,Madera,93610,2007-08-21,2025-01-01,37.131113,-120.242025,1.531352,77.12,2010-04-01,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2892.9075458251764,11.831222108044384,NaN,NaN,1.531352,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,0.0,0
4,2010.0,20651936023907,Madera,93610,1980-07-01,2025-01-01,37.115243,-120.267770,1.531352,70.66,2010-04-01,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2004.5665433091058,-57.72695665034661,NaN,NaN,1.531352,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0,0


In [221]:
#joined2.to_parquet("joined_datetime_replaced_10-10-22_2.parquet")

In [120]:
# joined2['school_open_date'] = joined2['school_open_date'].apply(change_dt_format)
# joined2['school_closed_date'] = joined2['school_closed_date'].apply(change_dt_format)
# joined2['year_month'] = joined2['year_month'].apply(change_dt_format)

In [28]:
# make date time back to a string
# it's 0 only if both of the dates were there in the first place

def check_school_open(year_month, school_open_date, school_closed_date):
    year_month = year_month
    school_open_date = school_open_date
    school_closed_date = school_closed_date
    try:
        if (school_open_date <= year_month) & (year_month <= school_closed_date):
            return 1
        else:
            return 0
    except:
        print("school open date is {}\ncurrent date is: {}\nschool close date is {}\n\n".format(school_open_date, year_month, school_closed_date))
    

In [29]:
joined['open_dates_valid'] = joined.apply(lambda row: check_school_open(row['year_month'], 
                                                                                    row['school_open_date'], 
                                                                                    row['school_closed_date']), axis = 1)

In [30]:
joined['open_dates_valid'].value_counts()

1    2471552
0     510688
Name: open_dates_valid, dtype: int64

In [31]:
joined.to_parquet("joined_schools_open_unfiltered_10-10-22.parquet")

In [32]:
joined2 = joined[joined['open_dates_valid'] == 1]
joined2.head()

,year,cdscode,school_county,school_zip,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,open_close_dates_valid,Taxable Year,Zip Code,CA_AGI_per_returns,Total Tax Liability,school_county_v2,school_region_name,nearby_point_source_count,open_dates_valid
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2010.0,20102070109728,Madera,93610,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04-01,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0,1
1,2010.0,20102072030187,Madera,93610,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04-01,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,1,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0,1
2,2010.0,20651930000000,Madera,93610,1990-01-01,2025-01-01,37.122449,-120.264870,1.531352,72.77,2010-04-01,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2358.38241919395,-37.54028477906731,NaN,NaN,1.531352,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,0,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,0.0,1
3,2010.0,20651930113076,Madera,93610,2007-08-21,2025-01-01,37.131113,-120.242025,1.531352,77.12,2010-04-01,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2892.9075458251764,11.831222108044384,NaN,NaN,1.531352,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,0.0,1
4,2010.0,20651936023907,Madera,93610,1980-07-01,2025-01-01,37.115243,-120.267770,1.531352,70.66,2010-04-01,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2004.5665433091058,-57.72695665034661,NaN,NaN,1.531352,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2,2010.0,93610,14375.028223,2905021.0,Madera,Northern San Joaquin Valley,1.0,1


In [33]:
joined2.shape[0]

2471552

In [34]:
joined2.to_parquet("joined_open_schools_only_10-10-22.parquet")

In [36]:
joined2.isnull().sum(axis=0)

year                              0
cdscode                           0
school_county                     0
school_zip                        0
school_open_date                  0
school_closed_date                0
school_lat                        0
school_lon                        0
school_wspd                       0
school_elevation_m                0
year_month                        0
total_population               1584
total_population_male          1584
total_population_female        1584
population_0_4                 1584
population_0_4_male            1584
population_0_4_female          1584
population_5_9                 1584
population_5_9_male            1584
population_5_9_female          1584
population_10_14               1584
population_10_14_male          1584
population_10_14_female        1584
population_15_19               1584
population_15_19_male          1584
population_15_19_female        1584
pop_under19_male               1584
pop_under19_female          

In [37]:
joined2.dtypes

year                                float64
cdscode                               int64
school_county                        object
school_zip                            int32
school_open_date             datetime64[ns]
school_closed_date           datetime64[ns]
school_lat                          float64
school_lon                          float64
school_wspd                         float64
school_elevation_m                  float64
year_month                   datetime64[ns]
total_population                     object
total_population_male                object
total_population_female              object
population_0_4                       object
population_0_4_male                  object
population_0_4_female                object
population_5_9                       object
population_5_9_male                  object
population_5_9_female                object
population_10_14                     object
population_10_14_male                object
population_10_14_female         

# Convert school open dates and school closed dates to datetime (old)

In [11]:
joined.replace(to_replace='No Data', value=np.nan, inplace=True)

In [12]:
joined[joined['school_open_date'] == 'No Data']

,year,cdscode,school_county,school_zip,school_region_name,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine


In [13]:
joined['school_open_date'] = pd.to_datetime(joined['school_open_date'], format='%MMDDYYYY')
#joined['school_open_date'] = joined['school_open_date'].dt.date

In [14]:
joined['school_closed_date'] = pd.to_datetime(joined['school_closed_date'], format='%MMDDYYYY')

In [15]:
#joined['new_year'] = pd.to_datetime(joined['year'], format='%Y')

In [20]:
joined['new_ym'] = pd.to_datetime(joined['year_month'])

In [21]:
joined.head()

,year,cdscode,school_county,school_zip,school_region_name,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,school_elevation_m,year_month,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female,pop_under19_male,pop_under19_female,total_pop_under19,pm25,point_source_lat,point_source_lon,ps_elevation_m,point_source_pm25_tpy,dist_school_to_ps_m,angle_to_school,ps_wind_lat,ps_wind_lon,ps_wspd_merge,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,new_ym
0,2010.0,20102070109728,Madera,93610,Northern San Joaquin Valley,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,2010-04-01
1,2010.0,20102072030187,Madera,93610,Northern San Joaquin Valley,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,70.84,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,1873.525780181077,-55.29643968357893,NaN,NaN,1.531352,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,2010-04-01
2,2010.0,20651930000000,Madera,93610,Northern San Joaquin Valley,NaT,NaT,37.122449,-120.264870,1.531352,72.77,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2358.38241919395,-37.54028477906731,NaN,NaN,1.531352,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,2010-04-01
3,2010.0,20651930113076,Madera,93610,Northern San Joaquin Valley,2007-08-21,NaT,37.131113,-120.242025,1.531352,77.12,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2892.9075458251764,11.831222108044384,NaN,NaN,1.531352,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2010-04-01
4,2010.0,20651936023907,Madera,93610,Northern San Joaquin Valley,1980-07-01,NaT,37.115243,-120.267770,1.531352,70.66,2010-04,23833.0,8280.0,15553.0,1439.0,694.0,745.0,1485.0,729.0,756.0,1407.0,740.0,667.0,1473.0,753.0,720.0,2916.0,2888.0,5804.0,7.736047,37.1056,-120.2487,73.3,2.0127727,2004.5665433091058,-57.72695665034661,NaN,NaN,1.531352,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2010-04-01


In [16]:
joined['year_month']

0          2010-04
1          2010-04
2          2010-04
3          2010-04
4          2010-04
            ...   
2983106    2016-06
2983107    2016-06
2983108    2016-06
2983109    2016-06
2983110    2016-06
Name: year_month, Length: 2983111, dtype: object

In [22]:
joined['new_ym']

0         2010-04-01
1         2010-04-01
2         2010-04-01
3         2010-04-01
4         2010-04-01
             ...    
2983106   2016-06-01
2983107   2016-06-01
2983108   2016-06-01
2983109   2016-06-01
2983110   2016-06-01
Name: new_ym, Length: 2983111, dtype: datetime64[ns]

In [23]:
type(joined['school_closed_date'][2])

pandas._libs.tslibs.nattype.NaTType

In [ ]:
# def new_school_open_close(year, school_open_date, school_closed_date):
#     year = year
#     school_open_date = school_open_date
#     school_closed_date = school_closed_date
#     if pd.isnull(school_closed_date):
#         new_school_closed = '1990-01-01'
#     elif pd.isnull(school_opened_date):
#         new_school_closed = '2025-01-01'

In [212]:
# make date time back to a string
# it's 0 only if both of the dates were there in the first place

# def check_school_open_close(year, school_open_date, school_closed_date):
#     year = year
#     school_open_date = school_open_date
#     school_closed_date = school_closed_date
#     try:
#         # if both missing, still 1
#         # 0 means both there and current date doesn't fall within
#         if (pd.isnull(school_open_date)==False) and (pd.isnull(school_closed_date)):
#             if year > school_open_date:
#                 return 2 # still 1
#             else:
#                 return 0
#         elif (pd.isnull(school_open_date)) and (pd.isnull(school_closed_date)==False):
#             return 3
#         elif pd.isnull(school_open_date) and pd.isnull(school_closed_date):
#             return 0
#         elif (year > school_open_date) & (year < school_closed_date):
#             return 1
#         else:
#             return 0
#     except:
#         print("school open date is {}\nschool close date is {}".format(school_open_date, school_closed_date))
    

In [26]:
joined['open_close_dates_valid'] = joined.apply(lambda row: check_school_open_close(row['new_ym'], 
                                                                                    row['school_open_date'], 
                                                                                    row['school_closed_date']), axis = 1)

In [126]:
# old ones joined only with year
#joined['open_close_dates_valid'].value_counts()
# 0 means both open and closed dates are missing, or that the year was before the school open date
# 1 means has both open and closed dates and year falls within that range (good)
# 2 means that only has open date but the year falls after the open date (good)

2    2054340
0     759919
1     168852
Name: open_close_dates_valid, dtype: int64

In [27]:
# new one joined with year-month
joined['open_close_dates_valid'].value_counts()

2    2064541
0     753378
1     165192
Name: open_close_dates_valid, dtype: int64

In [28]:
len(joined[pd.isnull(joined['school_open_date'])])

241280

In [29]:
len(joined[pd.isnull(joined['school_closed_date'])])

2628068

In [30]:
joined.shape[0]

2983111

In [31]:
joined.dtypes

year                                float64
cdscode                              object
school_county                        object
school_zip                           object
school_region_name                   object
school_open_date             datetime64[ns]
school_closed_date           datetime64[ns]
school_lat                          float64
school_lon                          float64
school_wspd                         float64
school_elevation_m                  float64
year_month                           object
total_population                     object
total_population_male                object
total_population_female              object
population_0_4                       object
population_0_4_male                  object
population_0_4_female                object
population_5_9                       object
population_5_9_male                  object
population_5_9_female                object
population_10_14                     object
population_10_14_male           

In [32]:
joined.to_parquet("joined_datetime_ym.parquet")

# Temperature

In [144]:
school_elevations.head()

,cdscode,school_lat,school_lon,elevation_meters
0,20102070109728,37.115210,-120.26603,70.84
1,20102072030187,37.115210,-120.26603,70.84
2,20651930000000,37.122449,-120.26487,72.77
3,20651930113076,37.131113,-120.242025,77.12
4,20651936023907,37.115243,-120.26777,70.66


In [159]:
school_elevations['school_lat'] = pd.to_numeric(school_elevations['school_lat'])
school_elevations['school_lon'] = pd.to_numeric(school_elevations['school_lon'])

In [160]:
school_elevations.dtypes

cdscode              object
school_lat          float64
school_lon          float64
elevation_meters    float64
dtype: object

In [161]:
joined_with_elevations = pd.merge(joined, school_elevations,  how='left', left_on=['cdscode', 'school_lat','school_lon'], right_on = ['cdscode', 'school_lat','school_lon'])

In [162]:
len(joined_with_elevations)

2983111

In [166]:
joined_with_elevations['elevation_meters'].isnull().sum()

0

In [177]:
len(joined_with_elevations['elevation_meters'])

2983111

In [167]:
joined_with_elevations.to_parquet("joined_datetime_elevations.parquet")

In [176]:
joined_with_elevations.dtypes

year                                float64
cdscode                              object
school_county                        object
school_zip                           object
school_region_name                   object
school_open_date             datetime64[ns]
school_closed_date           datetime64[ns]
school_lat                          float64
school_lon                          float64
school_wspd                         float64
school_elevation_m                  float64
year_month                           object
total_population                     object
total_population_male                object
total_population_female              object
population_0_4                       object
population_0_4_male                  object
population_0_4_female                object
population_5_9                       object
population_5_9_male                  object
population_5_9_female                object
population_10_14                     object
population_10_14_male           

In [178]:
joined_with_elevations.columns

Index(['year', 'cdscode', 'school_county', 'school_zip', 'school_region_name',
       'school_open_date', 'school_closed_date', 'school_lat', 'school_lon',
       'school_wspd', 'school_elevation_m', 'year_month', 'total_population',
       'total_population_male', 'total_population_female', 'population_0_4',
       'population_0_4_male', 'population_0_4_female', 'population_5_9',
       'population_5_9_male', 'population_5_9_female', 'population_10_14',
       'population_10_14_male', 'population_10_14_female', 'population_15_19',
       'population_15_19_male', 'population_15_19_female', 'pop_under19_male',
       'pop_under19_female', 'total_pop_under19', 'pm25', 'point_source_lat',
       'point_source_lon', 'ps_elevation_m', 'point_source_pm25_tpy',
       'dist_school_to_ps_m', 'angle_to_school', 'ps_wind_lat', 'ps_wind_lon',
       'ps_wspd_merge', 'school_wdir_wrt_0N', 'ps_wdir_wrt_0N',
       'school_wind_alignment', 'ps_wind_alignment', 'avg_wind_speed',
       'avg_wind_alig

In [181]:
joined_with_elevations['school_elevation_m']

0            70.84
1            70.84
2            72.77
3            77.12
4            70.66
            ...   
2983106    1794.66
2983107    1840.70
2983108    1794.66
2983109    1895.97
2983110    1895.72
Name: school_elevation_m, Length: 2983111, dtype: float64

In [183]:
joined_with_elevations.drop(columns=['elevation_meters'], inplace=True)

In [ ]:
def get_temp(year, lat, lon, elevation):
    year = year
    start = datetime(2018, 1, 1)
    end = datetime(2018, 1, 1)
    
    point = Point(lat, lon, elevation)

In [172]:
# Point(lat, lon, elevation)
start = datetime(2018, 1, 1)
end = datetime(2018, 12, 31)

vancouver = Point(37.105600, -120.248700, 73.30)

In [173]:
data = Daily(vancouver, start, end)
data = data.fetch()

In [175]:
data

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2018-01-01,8.2,1.1,17.2,0.0,NaN,NaN,3.3,NaN,1022.8,NaN
2018-01-02,9.4,5.6,15.0,0.0,NaN,NaN,3.1,NaN,1021.6,NaN
2018-01-03,10.9,4.4,17.2,0.0,NaN,NaN,7.8,NaN,1017.8,NaN
2018-01-04,14.3,9.4,20.0,3.7,NaN,NaN,10.8,NaN,1019.6,NaN
2018-01-05,15.5,10.6,22.2,0.0,NaN,NaN,7.3,NaN,1021.8,NaN
...,...,...,...,...,...,...,...,...,...,...
2018-12-26,7.9,3.3,14.4,0.0,NaN,NaN,3.4,NaN,1019.7,NaN
2018-12-27,6.7,2.2,13.3,0.0,NaN,NaN,7.3,NaN,NaN,NaN
2018-12-28,6.0,1.1,12.8,0.0,NaN,NaN,8.1,NaN,1023.0,NaN


In [171]:
data['tavg']

time
2018-01-01    8.2
Freq: D, Name: tavg, dtype: float64

In [66]:
# Set time period
start = datetime(2000, 1, 1)
end = datetime(2018, 12, 31)

#
#point = Point(37.105600,-120.248700,73.30)

# 3
point = Point(35.576700, -119.785400, 175.10)

# Get Monthly data
data = Monthly(point, start, end)
data = data.fetch()
data
#data = data.reset_index()


# Plot line chart including average, minimum and maximum temperature
#ata
# data.plot(y=['tavg', 'tmin', 'tmax'])
# plt.show()

,tavg,tmin,tmax,prcp,wspd,pres,tsun


In [35]:
data.reset_index()

,time,tavg,tmin,tmax,prcp,wspd,pres,tsun
0,2006-02-01,9.9,3.8,16.9,NaN,NaN,NaN,NaN
1,2006-03-01,9.5,4.4,14.7,NaN,NaN,NaN,NaN
2,2006-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2006-05-01,20.6,12.2,29.0,NaN,11.6,1013.2,NaN
4,2006-06-01,25.3,16.2,34.2,NaN,NaN,1012.1,NaN
...,...,...,...,...,...,...,...,...
149,2018-07-01,28.3,19.3,37.4,0.0,13.0,1012.4,NaN
150,2018-08-01,24.7,15.7,33.9,0.0,NaN,1012.1,NaN
151,2018-09-01,24.0,14.4,34.3,0.0,9.0,1011.1,NaN
152,2018-10-01,18.9,11.4,27.7,NaN,NaN,NaN,NaN


In [61]:
def get_temp(lat, lon, elevation):
    new_df_dict = {}
    year_start = 2000
    year_end = 2018
    month_start = 1
    month_end = 12
    lat = lat
    lon = lon
    elevation = elevation
    start = datetime(year_start, month_start, 31)
    end = datetime(year_end, month_end, 31)
    
    point = Point(lat, lon, elevation)
    month = Monthly(point, start, end)
    table = month.fetch()
    data = table.reset_index(drop=False)
    if data.empty:
        df_time = data['time']
    
    else:
        df_tavg = data['tavg']
        df_time = data['time']
        df_lat = pd.Series([lat])
        df_lat_r = df_lat.repeat(df_time.shape[0]).reset_index(drop=True)

        df_lon = pd.Series([lon])
        df_lon_r = df_lon.repeat(df_time.shape[0]).reset_index(drop=True)

        df_elevation = pd.Series([elevation])
        df_elevation_r = df_elevation.repeat(df_time.shape[0]).reset_index(drop=True)
    #idx = pd.Series(range(0,df_time.shape[0]))
    
    
    new_df = pd.concat([df_lat_r, df_lon_r, df_elevation_r, df_time, df_tavg], axis=1)
    return new_df
    
    #new_df_dict['point_source_lat'] = lat
    
    #point['year_month'] = point['time'][:7]

In [5]:
point_elevations_test = point_elevations[:5]

In [6]:
point_elevations_test

,point_source_lat,point_source_lon,elevation_meters
0,37.105600,-120.248700,73.30
1,35.151250,-118.016670,740.08
2,35.513841,-118.901299,250.90
3,35.576700,-119.785400,175.10
4,34.534084,-117.861722,900.09


In [62]:
testing_new_df = point_elevations_test.apply(lambda row: get_temp(row['point_source_lat'], 
                                                                  row['point_source_lon'], 
                                                                  row['elevation_meters']), axis=1)

          time  tavg  tmin  tmax  prcp  wspd    pres  tsun
0   2006-02-01   9.9   3.8  16.9   NaN   NaN     NaN   NaN
1   2006-03-01   9.5   4.4  14.7   NaN   NaN     NaN   NaN
2   2006-04-01   NaN   NaN   NaN   NaN   NaN     NaN   NaN
3   2006-05-01  20.6  12.2  29.0   NaN  11.6  1013.2   NaN
4   2006-06-01  25.3  16.2  34.2   NaN   NaN  1012.1   NaN
..         ...   ...   ...   ...   ...   ...     ...   ...
149 2018-07-01  28.3  19.3  37.4   0.0  13.0  1012.4   NaN
150 2018-08-01  24.7  15.7  33.9   0.0   NaN  1012.1   NaN
151 2018-09-01  24.0  14.4  34.3   0.0   9.0  1011.1   NaN
152 2018-10-01  18.9  11.4  27.7   NaN   NaN     NaN   NaN
153 2018-11-01  13.3   6.8  21.6   NaN   5.6     NaN   NaN

[154 rows x 8 columns]
          time  tavg  tmin  tmax  prcp  wspd    pres  tsun
0   2006-09-01  22.9  12.1  32.9   0.0  12.6  1012.0   NaN
1   2006-10-01   NaN   NaN   NaN   NaN   NaN     NaN   NaN
2   2006-11-01  11.1   1.7  21.3   NaN  11.8  1017.8   NaN
3   2006-12-01   NaN   NaN   NaN

KeyError: 'time'

In [34]:
point_elevations.head()

,point_source_lat,point_source_lon,elevation_meters
0,37.105600,-120.248700,73.30
1,35.151250,-118.016670,740.08
2,35.513841,-118.901299,250.90
3,35.576700,-119.785400,175.10
4,34.534084,-117.861722,900.09


In [40]:
len(point_elevations)

3330

In [35]:
school_elevations.head()

,cdscode,school_lat,school_lon,elevation_meters
0,20102070109728,37.115210,-120.26603,70.84
1,20102072030187,37.115210,-120.26603,70.84
2,20651930000000,37.122449,-120.26487,72.77
3,20651930113076,37.131113,-120.242025,77.12
4,20651936023907,37.115243,-120.26777,70.66


In [41]:
len(school_elevations)

13297